# Tarea 1

Santiago Martínez Novoa - 202112020

David Gonzalez Vargas - 202110240

In [75]:
import pandas as pd
import numpy as np
from typing import List

## Punto 1. Métricas de evaluación

### Precision

In [76]:
def precision(relevance_query:List[int]) -> float:
    """
    Calcula la precision de una consulta dada una lista de relevancia binaria
    Args:
        relevance_query (List[int]): Lista de relevancia binaria (1=relevante, 0=no relevante)
    Returns:
        float: Precision de la consulta
    """
    relevance_query = np.array(relevance_query)
    return np.mean(relevance_query)

In [77]:
relevance_query_1 = [0, 0, 0, 1]

print(precision(relevance_query_1))

0.25


### Precision @ K

In [78]:
def precision_at_k(relevance_query: List[int], k: int) -> float:
    """
    Calcula la precision en el top-k de una consulta dada una lista de relevancia binaria
    Args:
        relevance_query (List[int]): Lista de relevancia binaria (1=relevante, 0=no relevante)
        k (int): Número de elementos a considerar en el top-k
    Returns:
        float: Precision en el top-k de la consulta
    """
    if k <= 0:
        raise ValueError("k debe ser un entero positivo menor o igual al tamaño de la lista de relevancia")
    
    relevance_query = np.array(relevance_query)[:k]
    return np.mean(relevance_query)

In [79]:
relevance_query_2 = [0, 0, 0, 1]
k = 1
print(precision_at_k(relevance_query_2, k))

0.0


### Recall @ K

In [80]:
def recall_at_k(relevance_query: List[int], number_relevant_docs: int,  k: int) -> float:
    """
    Calcula la recall en el top-k de una consulta dada una lista de relevancia binaria
    Args:
        relevance_query (List[int]): Lista de relevancia binaria (1=relevante, 0=no relevante)
        number_relevant_docs (int): Número total de documentos relevantes
        k (int): Número de elementos a considerar en el top-k
    Returns:
        float: Recall en el top-k de la consulta
    """
    if k <= 0:
        raise ValueError("k debe ser un entero positivo menor o igual al tamaño de la lista de relevancia")
    if number_relevant_docs <= 0:
        raise ValueError("El número de documentos relevantes debe ser un entero positivo")
        
    relevance_query = np.array(relevance_query)[:k]
    return np.sum(relevance_query) / number_relevant_docs
    

In [81]:
relevance_query_3 = [0, 0, 0, 1]
k = 1
number_relevant_docs = 4

print(recall_at_k(relevance_query_3, number_relevant_docs, k))

0.0


### Average Precision

In [82]:
def average_precision(relevance_query: List[int]) -> float:
    """
    Calcula la precisión promedio de una consulta dada una lista de relevancia binaria
    Args:
        relevance_query (List[int]): Lista de relevancia binaria (1=relevante, 0=no relevante)
    Returns:
        float: Precisión promedio de la consulta
    """
    relevance_query = np.array(relevance_query)
    cumulative_precision = 0.0
    for i in range(len(relevance_query)):
        if relevance_query[i] == 1:
            cumulative_precision += precision_at_k(relevance_query, i + 1)
    if np.sum(relevance_query) == 0:
        return 0.0
    return cumulative_precision / np.sum(relevance_query)

In [83]:
relevance_query_4 = [0,1,0,1,1,1,1]
print(average_precision(relevance_query_4))

0.5961904761904762


### Mean Average Precision (mAP)

In [84]:
def mean_average_precision(relevance_queries: List[List[int]]) -> float:
    """
    Calcula la precisión promedio de un conjunto de consultas
    Args:
        relevance_queries (List[List[int]]): Lista de listas de relevancia binaria
    Returns:
        float: Precisión promedio de todas las consultas
    """
    return np.mean([average_precision(query) for query in relevance_queries])

In [91]:
queries = [
    [0, 1, 0, 1, 1, 1, 1],
    [1, 0, 1, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 1]
]
relevance_query_list = [
    [0, 1, 0, 1, 1, 1, 1],  # AP ≈ 0.5961904761904762
    [1, 0, 0, 0],           # AP = 1.0
    [0, 0, 0],              # AP = 0.0
]
print(mean_average_precision(relevance_query_list))
print(mean_average_precision(queries))

0.532063492063492
0.5764285714285714


### DCG @ K

In [86]:
def dcg_at_k(relevance_scores: List[int], k: int) -> float:
    """
    Calcula el Discounted Cumulative Gain (DCG) en la posición k
    Args:
        relevance_scores (List[int]): Lista de relevancia (1=relevante, 0=no relevante)
        k (int): La posición hasta la cual calcular el DCG
    Returns:
        float: El valor del DCG en la posición k
    """
    relevance_scores = np.array(relevance_scores)[:k]
    positions = np.arange(1, len(relevance_scores) + 1)
    discounts = np.log2(np.maximum(positions, 2))
    dgc = np.sum(relevance_scores / discounts)
    return dgc

In [87]:
relevance_query_3  = [4, 4, 3, 0, 0, 1, 3, 3, 3, 0]
k = 6
print(dcg_at_k(relevance_query_3, k))

10.279642067948915


### NDCG @ K

In [89]:
def ndcg_at_k(relevance_scores: List[int], k: int) -> float:
    """
    Calcula el Normalized Discounted Cumulative Gain (NDCG) en la posición k
    Args:
        relevance_scores (List[int]): Lista de relevancia (1=relevante, 0=no relevante)
        k (int): La posición hasta la cual calcular el NDCG
    Returns:
        float: El valor del NDCG en la posición k
    """
    dcg = dcg_at_k(relevance_scores, k)
    ideal_relevance = np.sort(relevance_scores)[::-1][:k]
    idcg = dcg_at_k(ideal_relevance, k)
    if idcg == 0:
        return 0.0
    return dcg / idcg

In [90]:
relevance_query_3  = [4, 4, 3, 0, 0, 1, 3, 3, 3, 0]
k = 6
print(ndcg_at_k(relevance_query_3, k))

0.7424602308163405
